In [1]:
import tweepy
from tweepy import Stream
import webbrowser
import time
import pandas as pd
!python -m pip install wordcloud
!python -m pip install textblob
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

You should consider upgrading via the '/Users/ruby/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ruby/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#one of the csv has a problem with the columns, the content of one column is another, and vice vera, so we need to take care of that first # reshuffle the data 
df = pd.read_csv('timeline_tweets_ukraine.csv')
df["temp"] = df['Time']
df["Time"] = df['Tweet']
df["Tweet"] = df["temp"]
del df["temp"]
df.to_csv('timeline_tweets_ukraine2.csv', index=False)


In [24]:
#merging  csv files into the first dataframe
#we already imported pandas up above
# merging multiple csv files

csv_files = [
    'putin_and_ukraine_tweets.csv',
    'timeline_tweets_ukraine2.csv',
    'timeline_tweets_putin.csv',
    'timeline_tweets_ukraine_2000.csv'
]
df_merged1 = pd.concat(map(pd.read_csv, csv_files), ignore_index=True)

print(df_merged1)

      Unnamed: 0             user  \
0              0       HerryNapit   
1              1       HerryNapit   
2              2       HerryNapit   
3              3     _aquariusmac   
4              4       HerryNapit   
...          ...              ...   
3585        1995      may_taratip   
3586        1996  Doungnapayard28   
3587        1997    KaYui04263359   
3588        1998         ampaikk1   
3589        1999  LJuNFWTzjlQzMSa   

                                                  Tweet  \
0     RT @HerryNapit: #Russia's invasion of #Ukraine...   
1     RT @HerryNapit: Lysychansk. The father saw his...   
2     RT @HerryNapit: Ukraine war: Putin creates glo...   
3     RT @AsianaTimes: #Putin is back on the world s...   
4     RT @HerryNapit: Russia: Did Putin try to poiso...   
...                                                 ...   
3585  RT @Shevchenk0_Mila: #Putin behaves like #Hitl...   
3586  RT @Shevchenk0_Mila: #Putin behaves like #Hitl...   
3587  RT @yungboosie56: 

In [25]:
df_merged1['Time']

0       2022-06-23 09:57:26+00:00
1       2022-06-23 09:56:56+00:00
2       2022-06-23 09:56:41+00:00
3       2022-06-23 09:56:30+00:00
4       2022-06-23 09:55:58+00:00
                  ...            
3585    2022-06-22 13:21:15+00:00
3586    2022-06-22 13:21:14+00:00
3587    2022-06-22 13:21:14+00:00
3588    2022-06-22 13:21:14+00:00
3589    2022-06-22 13:21:14+00:00
Name: Time, Length: 3590, dtype: object

In [26]:
#it does string replace and splicing

df_merged1['Time']= df_merged1['Time'].apply(lambda x:x.replace(':', '-')[:-6]) #last part is to remove the extra miliseconds
df_merged1['Time']

0       2022-06-23 09-57-26
1       2022-06-23 09-56-56
2       2022-06-23 09-56-41
3       2022-06-23 09-56-30
4       2022-06-23 09-55-58
               ...         
3585    2022-06-22 13-21-15
3586    2022-06-22 13-21-14
3587    2022-06-22 13-21-14
3588    2022-06-22 13-21-14
3589    2022-06-22 13-21-14
Name: Time, Length: 3590, dtype: object

In [27]:
print(df_merged1)

      Unnamed: 0             user  \
0              0       HerryNapit   
1              1       HerryNapit   
2              2       HerryNapit   
3              3     _aquariusmac   
4              4       HerryNapit   
...          ...              ...   
3585        1995      may_taratip   
3586        1996  Doungnapayard28   
3587        1997    KaYui04263359   
3588        1998         ampaikk1   
3589        1999  LJuNFWTzjlQzMSa   

                                                  Tweet                 Time  
0     RT @HerryNapit: #Russia's invasion of #Ukraine...  2022-06-23 09-57-26  
1     RT @HerryNapit: Lysychansk. The father saw his...  2022-06-23 09-56-56  
2     RT @HerryNapit: Ukraine war: Putin creates glo...  2022-06-23 09-56-41  
3     RT @AsianaTimes: #Putin is back on the world s...  2022-06-23 09-56-30  
4     RT @HerryNapit: Russia: Did Putin try to poiso...  2022-06-23 09-55-58  
...                                                 ...                  ...  
35

In [30]:
#column reshuffling #created.a new column and put all the values in the time column here
df_merged1["DateTimeStr"] = df_merged1['Time']
print(df_merged1)

      Unnamed: 0             user  \
0              0       HerryNapit   
1              1       HerryNapit   
2              2       HerryNapit   
3              3     _aquariusmac   
4              4       HerryNapit   
...          ...              ...   
3585        1995      may_taratip   
3586        1996  Doungnapayard28   
3587        1997    KaYui04263359   
3588        1998         ampaikk1   
3589        1999  LJuNFWTzjlQzMSa   

                                                  Tweet                 Time  \
0     RT @HerryNapit: #Russia's invasion of #Ukraine...  2022-06-23 09-57-26   
1     RT @HerryNapit: Lysychansk. The father saw his...  2022-06-23 09-56-56   
2     RT @HerryNapit: Ukraine war: Putin creates glo...  2022-06-23 09-56-41   
3     RT @AsianaTimes: #Putin is back on the world s...  2022-06-23 09-56-30   
4     RT @HerryNapit: Russia: Did Putin try to poiso...  2022-06-23 09-55-58   
...                                                 ...                  ..

In [32]:
from datetime import datetime
_format = '%Y-%m-%d %H-%M-%S'
dt_func = lambda dt_str: datetime.strptime(str(dt_str), _format) #convert datetime str to datetime obj
time_func = lambda dt_str: datetime.strptime(str(dt_str), _format).time() #convert datetime str to time
date_func = lambda dt_str: datetime.strptime(str(dt_str), _format).date() #convert datetime str to date
#df_merged1["DateTimeObj"]= df_merged1["DateTimeStr"].apply(dt_func)
df_merged1["Date"]= df_merged1["DateTimeStr"].apply(dt_func)
df_merged1["Time"]= df_merged1["DateTimeStr"].apply(dt_func)


In [18]:
print(df_merged1["Date"],"\n"*2)
print(df_merged1["Time"],"\n"*2)
#print(df_merged1["DateTimeObj"],"\n"*2)

0      2022-06-23 09:57:26
1      2022-06-23 09:56:56
2      2022-06-23 09:56:41
3      2022-06-23 09:56:30
4      2022-06-23 09:55:58
               ...        
3585   2022-06-22 13:21:15
3586   2022-06-22 13:21:14
3587   2022-06-22 13:21:14
3588   2022-06-22 13:21:14
3589   2022-06-22 13:21:14
Name: Date, Length: 3590, dtype: datetime64[ns] 


0      2022-06-23 09:57:26
1      2022-06-23 09:56:56
2      2022-06-23 09:56:41
3      2022-06-23 09:56:30
4      2022-06-23 09:55:58
               ...        
3585   2022-06-22 13:21:15
3586   2022-06-22 13:21:14
3587   2022-06-22 13:21:14
3588   2022-06-22 13:21:14
3589   2022-06-22 13:21:14
Name: Time, Length: 3590, dtype: datetime64[ns] 




In [33]:
df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   3590 non-null   int64         
 1   user         3590 non-null   object        
 2   Tweet        3590 non-null   object        
 3   Time         3590 non-null   datetime64[ns]
 4   DateTimeStr  3590 non-null   object        
 5   Date         3590 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 168.4+ KB


In [34]:
column = ['DateTimeStr']
df_merged1.drop(column, axis=1, inplace=True)
df_merged1

,Unnamed: 0,user,Tweet,Time,Date
0,0,HerryNapit,RT @HerryNapit: #Russia's invasion of #Ukraine...,2022-06-23 09:57:26,2022-06-23 09:57:26
1,1,HerryNapit,RT @HerryNapit: Lysychansk. The father saw his...,2022-06-23 09:56:56,2022-06-23 09:56:56
2,2,HerryNapit,RT @HerryNapit: Ukraine war: Putin creates glo...,2022-06-23 09:56:41,2022-06-23 09:56:41
3,3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
4,4,HerryNapit,RT @HerryNapit: Russia: Did Putin try to poiso...,2022-06-23 09:55:58,2022-06-23 09:55:58
...,...,...,...,...,...
3585,1995,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
3586,1996,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
3587,1997,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
3588,1998,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [35]:
df_merged1.drop(df_merged1.columns[[0]], axis=1, inplace=True)
df_merged1

,user,Tweet,Time,Date
0,HerryNapit,RT @HerryNapit: #Russia's invasion of #Ukraine...,2022-06-23 09:57:26,2022-06-23 09:57:26
1,HerryNapit,RT @HerryNapit: Lysychansk. The father saw his...,2022-06-23 09:56:56,2022-06-23 09:56:56
2,HerryNapit,RT @HerryNapit: Ukraine war: Putin creates glo...,2022-06-23 09:56:41,2022-06-23 09:56:41
3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
4,HerryNapit,RT @HerryNapit: Russia: Did Putin try to poiso...,2022-06-23 09:55:58,2022-06-23 09:55:58
...,...,...,...,...
3585,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
3586,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
3587,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
3588,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [39]:
df_merged1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   user    3590 non-null   object        
 1   Tweet   3590 non-null   object        
 2   Time    3590 non-null   datetime64[ns]
 3   Date    3590 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 112.3+ KB


In [43]:
df_merged1[50:100]

,user,Tweet,Time,Date
50,tameryazar,In the darkness of war they are also there...\...,2022-06-23 08:53:04,2022-06-23 08:53:04
51,KIAAcademYsir,💡DAY-116 ... Stop WAR &amp; Make PEACE ..🕊️\...,2022-06-23 08:52:37,2022-06-23 08:52:37
52,onesimohere,"Of course, we should not expect justice or com...",2022-06-23 08:52:33,2022-06-23 08:52:33
53,tameryazar,Local residents wait to receive humanitarian a...,2022-06-23 08:50:37,2022-06-23 08:50:37
54,caliophe20,"RT @anews: Parents in #Mariupol, #Ukraine are ...",2022-06-23 08:50:23,2022-06-23 08:50:23
55,TonyKen69354734,To every fool that thinks our Political Class ...,2022-06-23 08:48:39,2022-06-23 08:48:39
56,TheAccountantUK,RT @UxbEconomist07: Quick reminder of US #Infl...,2022-06-23 08:44:35,2022-06-23 08:44:35
57,Lx121,@b_chshevik @EyyRadioFree @gobloid3 3 things: ...,2022-06-23 08:42:05,2022-06-23 08:42:05
58,INSTINC26289171,RT @HerryNapit: Dictator Putin’s Afghanistan: ...,2022-06-23 08:40:00,2022-06-23 08:40:00
59,RiteBoss,RT @HerryNapit: Dictator Putin’s Afghanistan: ...,2022-06-23 08:39:59,2022-06-23 08:39:59


In [64]:
df_test = df_merged1.copy()

In [61]:
#drop duplicates
subsets = ['user', 'Tweet']
df_test.drop_duplicates(subset =subsets, keep = False, inplace = True, ignore_index= True)
df_test

,user,Tweet,Time,Date
0,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
1,Judgenap,Putin did NOT start the Ukraine War\nCol Doug ...,2022-06-21 19:42:58,2022-06-21 19:42:58
2,kiraincongress,The #USA is ready to hand #Ukraine all the wea...,2022-06-14 06:15:00,2022-06-14 06:15:00
3,BlackCo16122436,@BBCWorld .@BBCWorld Brutus Macron wants Putin...,2022-06-23 09:35:37,2022-06-23 09:35:37
4,BlackCo16122436,@BBCWorld .@BBCWorld Will #Berlusconi give Sic...,2022-06-23 09:35:23,2022-06-23 09:35:23
...,...,...,...,...
1293,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
1294,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
1295,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
1296,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [63]:
df_test['Tweet'].loc[1293] == df_test['Tweet'].loc[1297]

True

In [65]:
subsets = ['user', 'Tweet', 'Time', 'Date']
df_test.drop_duplicates(subset =subsets, keep = False, inplace = True, ignore_index= True)
df_test

,user,Tweet,Time,Date
0,HerryNapit,RT @HerryNapit: #Russia's invasion of #Ukraine...,2022-06-23 09:57:26,2022-06-23 09:57:26
1,HerryNapit,RT @HerryNapit: Lysychansk. The father saw his...,2022-06-23 09:56:56,2022-06-23 09:56:56
2,HerryNapit,RT @HerryNapit: Ukraine war: Putin creates glo...,2022-06-23 09:56:41,2022-06-23 09:56:41
3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
4,HerryNapit,RT @HerryNapit: Russia: Did Putin try to poiso...,2022-06-23 09:55:58,2022-06-23 09:55:58
...,...,...,...,...
1309,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
1310,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
1311,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
1312,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [67]:
df_test['Tweet'].loc[1309] == df_test['Tweet'].loc[1313]

True

In [53]:
df_test.drop_duplicates(keep = False, inplace = True, ignore_index= True)
df_test

,user,Tweet,Time,Date
0,HerryNapit,RT @HerryNapit: #Russia's invasion of #Ukraine...,2022-06-23 09:57:26,2022-06-23 09:57:26
1,HerryNapit,RT @HerryNapit: Lysychansk. The father saw his...,2022-06-23 09:56:56,2022-06-23 09:56:56
2,HerryNapit,RT @HerryNapit: Ukraine war: Putin creates glo...,2022-06-23 09:56:41,2022-06-23 09:56:41
3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
4,HerryNapit,RT @HerryNapit: Russia: Did Putin try to poiso...,2022-06-23 09:55:58,2022-06-23 09:55:58
...,...,...,...,...
1309,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
1310,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
1311,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
1312,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [56]:
df_test['Tweet'].loc[1309]

'RT @Shevchenk0_Mila: #Putin behaves like #Hitler, imitates him in every possible way.\nNow he is promoting hunger around the world.\nWe must…'

In [57]:
df_test['Tweet'].loc[1310]

'RT @Shevchenk0_Mila: #Putin behaves like #Hitler, imitates him in every possible way.\nNow he is promoting hunger around the world.\nWe must…'

In [58]:
df_test['Tweet'].loc[1311]

'RT @yungboosie56: #Putin behaves like #Hitler, imitates him in every possible way.\nNow he is promoting hunger around the world.\nWe must hel…'

In [59]:
df_test['Tweet'].loc[1309] == df_test['Tweet'].loc[1313]

True

In [46]:
#drop duplicates
subsets = ['user', 'Tweet']
df_test.drop_duplicates(subset =subsets,
                     keep = False, inplace = True)
df_test

,user,Tweet,Time,Date
3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
201,Judgenap,Putin did NOT start the Ukraine War\nCol Doug ...,2022-06-21 19:42:58,2022-06-21 19:42:58
202,kiraincongress,The #USA is ready to hand #Ukraine all the wea...,2022-06-14 06:15:00,2022-06-14 06:15:00
212,BlackCo16122436,@BBCWorld .@BBCWorld Brutus Macron wants Putin...,2022-06-23 09:35:37,2022-06-23 09:35:37
213,BlackCo16122436,@BBCWorld .@BBCWorld Will #Berlusconi give Sic...,2022-06-23 09:35:23,2022-06-23 09:35:23
...,...,...,...,...
3585,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
3586,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
3587,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
3588,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [48]:
df_test.head(50)

,user,Tweet,Time,Date
3,_aquariusmac,RT @AsianaTimes: #Putin is back on the world s...,2022-06-23 09:56:30,2022-06-23 09:56:30
201,Judgenap,Putin did NOT start the Ukraine War\nCol Doug ...,2022-06-21 19:42:58,2022-06-21 19:42:58
202,kiraincongress,The #USA is ready to hand #Ukraine all the wea...,2022-06-14 06:15:00,2022-06-14 06:15:00
212,BlackCo16122436,@BBCWorld .@BBCWorld Brutus Macron wants Putin...,2022-06-23 09:35:37,2022-06-23 09:35:37
213,BlackCo16122436,@BBCWorld .@BBCWorld Will #Berlusconi give Sic...,2022-06-23 09:35:23,2022-06-23 09:35:23
214,LinguaMaxillam,@campbellclaret #BorisJohnson is only the wors...,2022-06-23 09:33:58,2022-06-23 09:33:58
224,_Thirunarayan1,What to watch for in today's Jan. 6 panel hear...,2022-06-23 09:28:23,2022-06-23 09:28:23
225,_Thirunarayan1,Ohio State University wins trademark for the w...,2022-06-23 09:28:23,2022-06-23 09:28:23
226,_Thirunarayan1,New coronavirus subvariants escape antibodies ...,2022-06-23 09:28:23,2022-06-23 09:28:23
227,EDITORatWORK,#Russian #sanctions Links to RUSSIAN SANCTIONS...,2022-06-23 09:28:05,2022-06-23 09:28:05


In [49]:
df_test[1200:]

,user,Tweet,Time,Date
3492,Haleksandrony,"RT @jseldin: #Russia's war in #Ukraine "" It is...",2022-06-22 13:52:20,2022-06-22 13:52:20
3493,ModernEraNews,"RT jseldin: #Russia's war in #Ukraine "" It is ...",2022-06-22 13:52:08,2022-06-22 13:52:08
3494,InformationMaps,"RT @jseldin: #Russia's war in #Ukraine "" It is...",2022-06-22 13:51:36,2022-06-22 13:51:36
3495,C_W_UK,🇺🇦 #Ukraine #StandWithUkraine ✌\n#Putin https:...,2022-06-22 13:51:21,2022-06-22 13:51:21
3496,AceKaiser,"RT @Frentzhart2: Mein Gott, keine Waffen für d...",2022-06-22 13:50:57,2022-06-22 13:50:57
...,...,...,...,...
3585,may_taratip,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:15,2022-06-22 13:21:15
3586,Doungnapayard28,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14
3587,KaYui04263359,"RT @yungboosie56: #Putin behaves like #Hitler,...",2022-06-22 13:21:14,2022-06-22 13:21:14
3588,ampaikk1,RT @Shevchenk0_Mila: #Putin behaves like #Hitl...,2022-06-22 13:21:14,2022-06-22 13:21:14


In [ ]:
'RT @yungboosie56: #Putin behaves like #Hitler, imitates him in every possible way.\nNow he is promoting hunger around the world.\nWe must hel…'

In [ ]:
# my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
# my_url = 'https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/'

In [ ]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

In [ ]:
my_regex_pattern = r'

In [ ]:
df['tweet_new']=df['Tweet'].str.extract('(^[\d]{1,4})')
df


In [ ]:
def func(time_str):
    try:
        result = datetime.strptime(str(time_str), '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S')
    except:
        result = datetime.strptime(str(time_str), '%H-%M-%S').strftime('%H:%M:%S')
    finally:
        return result 

In [ ]:
#cleaningdataframe1,handling duplicates,dropping columns, standardizing capitalisation, and converting dtatypes
#dfmerged1.describe()
#dfmerged1.dtypes- this gave user, tweet and time as object, we gotta change time column
#dfmerged1.info()

#dfmerged1['Time'] = list(map(lambda x: datetime.datetime.strptime(x,'%b %d, %Y').strftime('%m/%d/%Y'), old_df['oldDate']))

dfmerged1['Time'] = dfmerged1['Time'].apply(lambda x:x[:-6])

#dfmerged1['Time'] = datetime.datetime.strptime(dfmerged1['Time'], '%a %b %d %Y').strftime('%d/%m/%Y')
#dfmerged1['Time'] = dfmerged1['Time'].str.replace(' (\d{1})', ' 0\\1')
dfmerged1['Time'] = pd.to_datetime(dfmerged1['Time'], format='%Y%m%d%H%M%S') #%Y%m%d%H%M%S #%H:%M:%S
#dfmerged1['Time'] = dfmerged1['Time'].astype('datetime64')

print(dfmerged1['Time'])

In [ ]:
#to seperate the time column intotime and date, as you can see they are still together
for d in df_merged1['Time']:
   df_merged1['Time'] = d.date()
   df_merged1['Date'] = d.time()
print(df_merged1)